# Exploring Futures Contracts and Market Data

This notebook demonstrates how to explore the domain objects.

In [ ]:
import sys
sys.path.insert(0, '/root/sw1')

from datetime import date
import pandas as pd
import matplotlib.pyplot as plt

from src.data.handler import DataHandler

## 1. Load Data Handler

In [ ]:
handler = DataHandler.from_processed_data('/root/sw1/processed_data', 'IC')
chain = handler.contract_chain
index = handler.index

## 2. Explore Index Data

In [ ]:
# Get index NAV series
nav = index.get_nav_series()
nav.plot(figsize=(12, 4), title=f'{index.name} NAV')
plt.show()

## 3. Explore Contracts

In [ ]:
# List all contracts
contracts = chain.get_all_contracts()
print(f"Total contracts: {len(contracts)}")

# Show first 10
for c in contracts[:10]:
    print(f"  {c.ts_code}: {c.list_date} to {c.delist_date}, mult={c.multiplier}")

In [ ]:
# Get a specific contract
contract = chain.get_contract('IC2312.CFX')
if contract:
    print(f"Contract: {contract}")
    print(f"Trading dates: {len(contract.get_trading_dates())}")
    
    # Get its price series
    dates = contract.get_trading_dates()
    prices = [contract.get_price(d, 'settle') for d in dates]
    pd.Series(prices, index=dates).plot(title=f'{contract.ts_code} Settlement Price')
    plt.show()

## 4. Get Active Contracts on a Date

In [ ]:
trade_date = date(2024, 6, 3)
active = chain.get_active_contracts(trade_date)

print(f"Active contracts on {trade_date}:")
for c in active:
    days = c.days_to_expiry(trade_date)
    vol = c.get_volume(trade_date)
    print(f"  {c.ts_code}: {days} days to expiry, volume={vol:.0f}")

## 5. Calculate Basis

In [ ]:
# Get snapshot for a date
snapshot = handler.get_snapshot(trade_date)

print(f"Index close: {snapshot.get_index_close():.2f}")

for ts_code in snapshot.get_available_contracts():
    basis = snapshot.get_basis(ts_code, relative=True)
    price = snapshot.get_futures_price(ts_code, 'settle')
    if basis is not None:
        print(f"  {ts_code}: price={price:.2f}, basis={basis:.4f} ({basis*100:.2f}%)")

## 6. Historical Basis Analysis

In [ ]:
# Calculate main contract basis over time
calendar = handler.get_trading_calendar(date(2020, 1, 1), date(2024, 12, 31))

basis_history = []
for d in calendar:
    main = chain.get_main_contract(d, rule='volume')
    if main:
        snapshot = handler.get_snapshot(d)
        basis = snapshot.get_basis(main.ts_code, relative=True)
        if basis is not None:
            basis_history.append({'date': d, 'basis': basis, 'contract': main.ts_code})

basis_df = pd.DataFrame(basis_history)
basis_df.set_index('date')['basis'].plot(figsize=(12, 4), title='Main Contract Basis Over Time')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.ylabel('Basis (F-S)/S')
plt.show()